In [ ]:
import pandas as pd

# Load the dataset from the specified location
file_path = 'data/HeartStepsV1-main/data_files/suggestions.csv'
columns_to_include = [
    'user.index', 'sugg.decision.utime','sugg.response.utime', 'sugg.device.since', 
    'interaction.count', 'send', 'send.sedentary', 'send.active', 'recognized.activity', 
    'tag.active', 'tag.indoor', 'tag.outdoor', 'tag.outdoor_snow', 'front.end.application', 
    'dec.location.category', 'dec.weather.condition', 'dec.temperature', 'dec.windspeed', 
    'dec.precipitation.chance', 'dec.snow', 'jbsteps60', 'jbmins60', 'response'
]

# Load the dataset with the specified columns and filter where 'Send' is True
suggestions_df = pd.read_csv(file_path, usecols=columns_to_include)


: 

In [1]:
import numpy as np

# Ensure 'sugg.decision.utime' is in datetime format
suggestions_df['sugg.decision.utime'] = pd.to_datetime(suggestions_df['sugg.decision.utime'])

# Step 1: Find rows where both 'dec.weather.condition' and 'dec.temperature' are NaN
rows_to_impute = suggestions_df[suggestions_df['dec.snow'].isna() & suggestions_df['dec.temperature'].isna()]

# Step 2: Impute missing values by finding the closest match
for idx, row in rows_to_impute.iterrows():
    user_id = row['user.index']
    decision_time = row['sugg.decision.utime']

    # Find rows from the same user that have non-NaN values for both weather and temperature
    potential_matches = suggestions_df[
        (suggestions_df['user.index'] == user_id) &
        (~suggestions_df['dec.snow'].isna()) &
        (~suggestions_df['dec.temperature'].isna())
    ]

    # Filter by rows within 6 hours of the current row's decision time
    time_difference = np.abs(potential_matches['sugg.decision.utime'] - decision_time)
    within_time_window = potential_matches[time_difference <= pd.Timedelta(hours=6)]

    if not within_time_window.empty:
        # Find the row with the smallest time difference
        closest_row = within_time_window.loc[time_difference.idxmin()]
        print(closest_row["sugg.decision.utime"], decision_time, closest_row["user.index"], user_id)

        # Step 3: Impute the missing values with those from the closest row
        suggestions_df.at[idx, 'dec.snow'] = closest_row['dec.snow']
        suggestions_df.at[idx, 'dec.temperature'] = closest_row['dec.temperature']

# Now, the missing values for 'dec.weather.condition' and 'dec.temperature' should be imputed where possible


NameError: name 'pd' is not defined

In [369]:
import numpy as np

# Ensure 'sugg.decision.utime' is in datetime format
suggestions_df['sugg.decision.utime'] = pd.to_datetime(suggestions_df['sugg.decision.utime'])

# Step 1: Find rows where both 'dec.weather.condition' and 'dec.temperature' are NaN
rows_to_impute = suggestions_df[suggestions_df['dec.weather.condition'].isna()]

# Step 2: Impute missing values by finding the closest match
for idx, row in rows_to_impute.iterrows():
    user_id = row['user.index']
    decision_time = row['sugg.decision.utime']

    # Find rows from the same user that have non-NaN values for both weather and temperature
    potential_matches = suggestions_df[
        (suggestions_df['user.index'] == user_id) &
        (~suggestions_df['dec.weather.condition'].isna()) 
    ]

    # Filter by rows within 6 hours of the current row's decision time
    time_difference = np.abs(potential_matches['sugg.decision.utime'] - decision_time)
    within_time_window = potential_matches[time_difference <= pd.Timedelta(hours=6)]

    if not within_time_window.empty:
        # Find the row with the smallest time difference
        closest_row = within_time_window.loc[time_difference.idxmin()]
        print(closest_row["sugg.decision.utime"], decision_time, closest_row["user.index"], user_id)

        # Step 3: Impute the missing values with those from the closest row
        suggestions_df.at[idx, 'dec.weather.condition'] = closest_row['dec.weather.condition']

# Now, the missing values for 'dec.weather.condition' and 'dec.temperature' should be imputed where possible


2015-08-30 21:31:00 2015-08-30 23:31:30 1 1
2015-07-24 17:01:56 2015-07-24 19:01:30 2 2
2015-07-25 12:31:44 2015-07-25 17:01:30 2 2
2015-07-25 17:01:30 2015-07-25 19:01:30 2 2
2015-07-28 22:01:20 2015-07-29 00:31:30 2 2
2015-07-31 00:32:02 2015-07-30 22:01:30 2 2
2015-07-31 12:34:48 2015-07-31 17:01:30 2 2
2015-07-31 17:01:30 2015-07-31 19:01:30 2 2
2015-08-01 17:01:32 2015-08-01 12:31:30 2 2
2015-08-02 19:00:57 2015-08-02 17:01:30 2 2
2015-08-03 00:32:56 2015-08-02 22:03:20 2 2
2015-08-03 22:02:22 2015-08-03 17:01:30 2 2
2015-08-03 17:01:30 2015-08-03 19:01:30 2 2
2015-08-03 22:02:22 2015-08-04 00:31:30 2 2
2015-08-06 17:01:31 2015-08-06 13:31:30 2 2
2015-08-07 00:33:29 2015-08-06 22:01:30 2 2
2015-08-07 19:02:47 2015-08-07 17:01:30 2 2
2015-08-08 19:02:42 2015-08-08 17:01:30 2 2
2015-08-12 00:32:05 2015-08-11 22:01:30 2 2
2015-08-12 17:01:19 2015-08-12 19:01:30 2 2
2015-08-15 00:31:05 2015-08-14 22:01:30 2 2
2015-08-15 19:01:43 2015-08-15 17:01:30 2 2
2015-08-17 19:01:15 2015-08-17 1

In [370]:
filtered_suggestions_df = suggestions_df[suggestions_df['send'] == True]
filtered_suggestions_df.shape

(3918, 23)

In [371]:
# Step 2: Handle missing sugg.response.utime by replacing with a placeholder and adjusting related columns
placeholder_response_time = "No Response"  # Define a placeholder value

# Ensure that when sugg.response.utime is NaN, interaction.count and response are NaN as well
missing_response = suggestions_df['sugg.response.utime'].isna() & suggestions_df['interaction.count'].isna() & suggestions_df['response'].isna()

# Replace NaN in sugg.response.utime with placeholder, set interaction.count to -1, and response to "No Response"
filtered_suggestions_df.loc[missing_response, 'sugg.response.utime'] = placeholder_response_time
filtered_suggestions_df.loc[missing_response, 'interaction.count'] = -1
filtered_suggestions_df.loc[missing_response, 'response'] = "No Response"


In [372]:
placeholder_response_time = "Unknown"  # Define a placeholder value

# Ensure that when sugg.response.utime is NaN, interaction.count and response are NaN as well
missing_response = suggestions_df['sugg.response.utime'].isna() & suggestions_df['interaction.count'].isna() & suggestions_df['response'].isna()

# Replace NaN in sugg.response.utime with placeholder, set interaction.count to -1, and response to "No Response"
filtered_suggestions_df.loc[missing_response, 'sugg.response.utime'] = placeholder_response_time
filtered_suggestions_df.loc[missing_response, 'interaction.count'] = -1
filtered_suggestions_df.loc[missing_response, 'response'] = "No Response"

In [373]:
missing_response = suggestions_df['sugg.device.since'].isna()

# Replace NaN in sugg.device.utime with placeholder, set sugg.device.since to 0
filtered_suggestions_df.loc[missing_response, 'sugg.device.since'] = 0

In [374]:
# Step 3: Drop rows where interaction.count or response are still NaN
filtered_suggestions_df.dropna(subset=['interaction.count', 'response'], inplace=True)
filtered_suggestions_df.dropna(subset=['jbmins60'], inplace=True)

filtered_suggestions_df.shape

/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/3236427295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_suggestions_df.dropna(subset=['interaction.count', 'response'], inplace=True)
/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/3236427295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_suggestions_df.dropna(subset=['jbmins60'], inplace=True)


(3439, 23)

In [375]:
# Step 4: Delete the column 'send'
filtered_suggestions_df.drop(columns=['send'], inplace=True)
filtered_suggestions_df.dropna(subset=['send.active'], inplace=True)
filtered_suggestions_df.shape


/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/3729160218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_suggestions_df.drop(columns=['send'], inplace=True)
/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/3729160218.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_suggestions_df.dropna(subset=['send.active'], inplace=True)


(3438, 22)

In [376]:
# Step 5: Convert NaN in front.end.application to "OFF"
filtered_suggestions_df['front.end.application'].fillna("OFF", inplace=True)
filtered_suggestions_df.shape




/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/2187487078.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_suggestions_df['front.end.application'].fillna("OFF", inplace=True)
/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/2187487078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sug

(3438, 22)

In [377]:
# Step 6: Create a new 'Time of Day' column from sugg.decision.utime
filtered_suggestions_df['sugg.decision.utime'] = pd.to_datetime(filtered_suggestions_df['sugg.decision.utime'])
filtered_suggestions_df['Time of Day'] = filtered_suggestions_df['sugg.decision.utime'].dt.time


/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/2843654072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_suggestions_df['sugg.decision.utime'] = pd.to_datetime(filtered_suggestions_df['sugg.decision.utime'])
/var/folders/qc/46z_x3td2l55q77fsffm737h0000gn/T/ipykernel_94965/2843654072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_suggestions_df['Time of Day'] = filtered_suggestions_df['sugg.decision.utime'].dt.time


In [378]:
nan_report = filtered_suggestions_df.isna().sum()
nan_report = nan_report[nan_report > 0]  # Only show columns with NaN values

if not nan_report.empty:
    print("Columns with NaN values and their counts:")
    print(nan_report)

    # For each column with NaN values, print the indices of the rows where NaN is present
    for column in nan_report.index:
        nan_indices = filtered_suggestions_df[filtered_suggestions_df[column].isna()].index.tolist()
        print(f"Indices with NaN in {column}: {nan_indices}")
else:
    print("No NaN values remaining in the dataset.")

Columns with NaN values and their counts:
dec.weather.condition     3
dec.temperature          26
dec.snow                 26
dtype: int64
Indices with NaN in dec.weather.condition: [3886, 4186, 5075]
Indices with NaN in dec.temperature: [1272, 1544, 1564, 1565, 1567, 1568, 1570, 1571, 1573, 1577, 1578, 1595, 1596, 1597, 1601, 1603, 2752, 2754, 2756, 3546, 3767, 3769, 3770, 3802, 3803, 3805]
Indices with NaN in dec.snow: [1272, 1544, 1564, 1565, 1567, 1568, 1570, 1571, 1573, 1577, 1578, 1595, 1596, 1597, 1601, 1603, 2752, 2754, 2756, 3546, 3767, 3769, 3770, 3802, 3803, 3805]


In [380]:
filtered_suggestions_df.to_csv("data/modified_data/Cleaned Dataset.csv")